# MaLSTM on Kaggle's Quora Question Pairs

This notebook is about implementing the MaLSTM model (http://www.mit.edu/~jonasm/info/MuellerThyagarajan_AAAI16.pdf) on Kaggle's Quora Question Pairs data.

Blog post containing a broader explanation about the network can be found in the following link https://medium.com/@eliorcohen/implementing-malstm-on-kaggles-quora-question-pairs-competition-8b31b0b16a07


# CODE

First, lets import all the necessary packages

In [6]:
import sys
print(sys.executable)

C:\Users\Korisnik\envs\chatbot2\Scripts\python.exe


In [11]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda, Dropout
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

Global variables

In [12]:
import os
cwd = os.getcwd()

In [13]:
# File paths
TRAIN_CSV = r"products_weights_vocabs/train.csv"
TEST_CSV = r"products_weights_vocabs/test.csv"
EMBEDDING_FILE = r"products_weights_vocabs/GoogleNews-vectors-negative300.bin"
MODEL_SAVING_DIR = os.path.join(cwd, 'model')

Create embedding matrix

In [10]:
# Load training and test set
train_df = pd.read_csv(TRAIN_CSV, low_memory=False)
test_df = pd.read_csv(TEST_CSV, low_memory=False)

stops = set(stopwords.words('english'))

In [13]:
def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

# Prepare embedding
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

questions_cols = ['question1', 'question2']

# Iterate over the questions only of both training and test datasets
for dataset in [train_df, test_df]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):

                # Check for unwanted words
                if word in stops or word not in word2vec.key_to_index:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.at[index, question] = q2n
            
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.key_to_index:
        embeddings[index] = word2vec[word]

del word2vec

In [11]:
"""
with open('vocabulary.pkl', 'wb') as f:
    pickle.dump(vocabulary, f)

with open('inverse_vocabulary.pkl', 'wb') as f:
    pickle.dump(inverse_vocabulary, f)
    
# Save embeddings
np.save('embeddings.npy', embeddings)
"""

"\nwith open('vocabulary.pkl', 'wb') as f:\n    pickle.dump(vocabulary, f)\n\nwith open('inverse_vocabulary.pkl', 'wb') as f:\n    pickle.dump(inverse_vocabulary, f)\n    \n# Save embeddings\nnp.save('embeddings.npy', embeddings)\n"

In [12]:
# embeddings = np.load('products_weights_vocabs/embeddings.npy')

Prepare training and validation data

In [14]:
max_seq_length = max(train_df.question1.map(lambda x: len(x)).max(),
                     train_df.question2.map(lambda x: len(x)).max(),
                     test_df.question1.map(lambda x: len(x)).max(),
                     test_df.question2.map(lambda x: len(x)).max())

# Split to train validation
validation_size = 40000
training_size = len(train_df) - validation_size

X = train_df[questions_cols]
Y = train_df['is_duplicate']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

# Split to dicts
X_train = {'left': X_train.question1, 'right': X_train.question2}
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}
X_test = {'left': test_df.question1, 'right': test_df.question2}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

### Build the model

In [15]:
# Model variables
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 25
dropout_rate = 0.2

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return tf.math.exp(-tf.math.reduce_sum(tf.math.abs(left - right), axis=1, keepdims=True))


# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden, dropout=dropout_rate, recurrent_dropout=dropout_rate)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

# Start training
training_start_time = time()

malstm_trained = malstm.fit(
    x=[X_train['left'], X_train['right']],
    y=Y_train,
    batch_size=batch_size,
    epochs=n_epoch,  # Use 'epochs' instead of 'nb_epoch'
    validation_data=([X_validation['left'], X_validation['right']], Y_validation)
)

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

C:\Users\Korisnik\envs\chatbot2\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/25
5693/5693 ━━━━━━━━━━━━━━━━━━━━ 267s 47ms/step - accuracy: 0.4353 - loss: 0.4417 - val_accuracy: 0.3892 - val_loss: 0.6083
Epoch 2/25
5693/5693 ━━━━━━━━━━━━━━━━━━━━ 281s 49ms/step - accuracy: 0.5720 - loss: 0.2549 - val_accuracy: 0.3892 - val_loss: 0.6084
Epoch 3/25
5693/5693 ━━━━━━━━━━━━━━━━━━━━ 276s 48ms/step - accuracy: 0.5870 - loss: 0.2493 - val_accuracy: 0.3892 - val_loss: 0.6084
Epoch 4/25
5693/5693 ━━━━━━━━━━━━━━━━━━━━ 283s 50ms/step - accuracy: 0.5947 - loss: 0.2460 - val_accuracy: 0.3892 - val_loss: 0.6085
Epoch 5/25
5693/5693 ━━━━━━━━━━━━━━━━━━━━ 273s 48ms/step - accuracy: 0.6049 - loss: 0.2429 - val_accuracy: 0.3892 - val_loss: 0.6085
Epoch 6/25
5693/5693 ━━━━━━━━━━━━━━━━━━━━ 284s 50ms/step - accuracy: 0.6074 - loss: 0.2420 - val_accuracy: 0.3892 - val_loss: 0.6086
Epoch 7/25
5693/5693 ━━━━━━━━━━━━━━━━━━━━ 278s 49ms/step - accuracy: 0.6127 - loss: 0.2406 - val_accuracy: 0.3892 - val_loss: 0.6086
Epoch 8/25
5693/5693 ━━━━━━━━━━━━━━━━━━━━ 282s 49ms/step - accuracy: 

### Results

In [2]:
#malstm.save('malstm_model.h5')
#malstm.save('malstm_model.keras')
#malstm.save_weights('malstm_weights.weights.h5')

In [ ]:
# Plot accuracy
plt.plot(malstm_trained.history['accuracy'])
plt.plot(malstm_trained.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()